# 4.2.2 各年度の企業別の単語数の統計
次に全 2360 件の各企業の文字数の統計情報を確認する。

In [2]:
import glob

def call_sample_dir_name(initial_name):
    if initial_name == "a":
        return "AfterSample"
    elif  initial_name == "t":
        return "TransitionPeriodSample"
    else:
        return "BeforeSample"


def call_csv_files(sample_dir_name="AfterSample", data_frame_spec=None, industry_spec=None):
    
    if data_frame_spec is None:
        
        if industry_spec is None:
            csv_files = glob.glob('/home/jovyan/3FetchingMDandA' + f"/**/{sample_dir_name}/*.csv", recursive=True)
        else:
             csv_files = glob.glob(f'/home/jovyan/3FetchingMDandA' + f"/**/{industry_spec}/{sample_dir_name}/*.csv", recursive=True)
    else:
        if industry_spec is None:
            csv_files = glob.glob(f'/home/jovyan/3FetchingMDandA/{data_frame_spec}' + f"/**/{sample_dir_name}/*.csv", recursive=True)
        else:
             csv_files = glob.glob(f"/home/jovyan/3FetchingMDandA/{data_frame_spec}/{industry_spec}/{sample_dir_name}/*.csv", recursive=True)
    
    return  csv_files

In [3]:
a_files =  call_csv_files(sample_dir_name="AfterSample")
print(len(a_files))
t_files =  call_csv_files(sample_dir_name="TransitionPeriodSample")
print(len(t_files))
b_files =  call_csv_files(sample_dir_name="BeforeSample")
print(len(b_files))

2277
2277
2277


In [4]:
import MeCab
import glob
import pandas as pd
import os 
import oseti



def calc_words_len(atb_file):
    
    atb_df = pd.read_csv(atb_file, index_col=0)
    if len(atb_df) < 1:
        return 0
    texts_joined = "".join(list(atb_df["Text"].values))
    #parse error対策
    texts_joined = texts_joined.replace("\n", "")
    return len(texts_joined)

In [5]:
data_frame_spec=None
industry_spec=None

dir_name_b = call_sample_dir_name("b")
before_csv_files  = call_csv_files(dir_name_b, data_frame_spec, industry_spec)

dir_name_t = call_sample_dir_name("t")
transition_period_csv_files = call_csv_files(dir_name_t, data_frame_spec, industry_spec)

dir_name_a = call_sample_dir_name("a")
after_csv_files = call_csv_files(dir_name_a, data_frame_spec, industry_spec)
print("--------ここまで終わりました1------")

--------ここまで終わりました1------


In [6]:
sum_words_before = []

for b_file in  before_csv_files :
    sum_words = calc_words_len(b_file) 
    sum_words_before.append(sum_words)
    
print("-------ここまで終わりました2-------")

-------ここまで終わりました2-------


In [7]:
sum_words_transition_period = []

for t_file in  transition_period_csv_files  :
    sum_words = calc_words_len(t_file) 
    sum_words_transition_period.append(sum_words)
print("-------ここまで終わりました3-------")

-------ここまで終わりました3-------


In [8]:
sum_words_after = []

for a_file in  after_csv_files :
    sum_words = calc_words_len(a_file) 
    sum_words_after.append(sum_words)
print("-------ここまで終わりました4-------")

-------ここまで終わりました4-------


In [9]:
year_li = []
for li in [sum_words_before, sum_words_transition_period, sum_words_after] :
    year_li.append(list(map(lambda x: 0 if x is None else x, li)))

In [10]:
df= pd.DataFrame(
    {
        "コロナ前":  year_li[0], 
        "コロナ過渡期":    year_li[1], 
        "コロナ後":  year_li[2], 
    })
df

,コロナ前,コロナ過渡期,コロナ後
0,21235,21390,9880
1,3877,3626,5835
2,4555,4580,4467
3,3780,4235,5049
4,11313,5352,6104
...,...,...,...
2272,10096,22454,13578
2273,22948,8026,20712
2274,10608,12077,6669
2275,11154,12411,7997


In [11]:
df.describe()

,コロナ前,コロナ過渡期,コロナ後
count,2277.000000,2277.000000,2277.000000
mean,8654.052701,8818.060606,9552.742644
std,5008.822989,4888.943032,4439.860706
min,944.000000,544.000000,2390.000000
25%,5667.000000,5852.000000,6769.000000
50%,7430.000000,7664.000000,8606.000000
75%,10033.000000,10204.000000,11133.000000
max,58434.000000,65264.000000,64800.000000


# 4.2.3 各年度の企業別の単語数の有意差の検証
ここでさらに追加で、年度ごとで単語数に有意差があるかについてもみていく。ここでは正規性や等分散性の検定を行なったのち、その後差の検定を行う。

In [12]:
before_li, transition_period_li, after_li = year_li[0],  year_li[1], year_li[2]

In [13]:
# 仮説
# https://analysis-navi.com/?p=3302
import scipy.stats as stats
print(stats.shapiro(before_li))   
print(stats.kstest(before_li, "norm"))

ShapiroResult(statistic=0.7526571750640869, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)


In [14]:
print(stats.shapiro(transition_period_li))   
print(stats.kstest(transition_period_li, "norm"))

ShapiroResult(statistic=0.7466043829917908, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)


In [15]:
print(stats.shapiro(after_li))   
print(stats.kstest(after_li, "norm"))

ShapiroResult(statistic=0.780049204826355, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)


In [16]:
#F検定の定義
import numpy as np
def exec_f_test(A, B):
    A_var = np.var(A, ddof=1)  # Aの不偏分散
    B_var = np.var(B, ddof=1)  # Bの不偏分散
    A_df = len(A) - 1  # Aの自由度
    B_df = len(B) - 1  # Bの自由度
    f = A_var / B_var  # F比の値
    one_sided_pval1 = stats.f.cdf(f, A_df, B_df)  # 片側検定のp値 1
    one_sided_pval2 = stats.f.sf(f, A_df, B_df)   # 片側検定のp値 2
    two_sided_pval = min(one_sided_pval1, one_sided_pval2) * 2  # 両側検定のp値
    print('F:       ', round(f, 3))
    print('p-value: ', round(two_sided_pval, 3))

In [17]:
#正規性の検定
  #before, transition    
A=before_li
B=transition_period_li
exec_f_test(A, B)

#ウェルチのt検定
    #before, transition
stats.ttest_ind(A, B, equal_var=False)

F:        1.05
p-value:  0.248


Ttest_indResult(statistic=-1.118129895035891, pvalue=0.2635705596333606)

In [19]:
#正規性の検定及びt検定
    #transition, after
A=transition_period_li
B=after_li
exec_f_test(A, B)

#スチューデントのt検定
    #transition, after
stats.ttest_ind(transition_period_li, after_li, axis=0, equal_var=True, nan_policy='propagate')

F:        1.213
p-value:  0.0


Ttest_indResult(statistic=-5.3084452807999245, pvalue=1.1582414092552812e-07)

In [20]:
import numpy as np
A=before_li
B=after_li
exec_f_test(A, B)

#スチューデントのt検定
    #transition, after
stats.ttest_ind(transition_period_li, after_li, axis=0, equal_var=True, nan_policy='propagate')

F:        1.273
p-value:  0.0


Ttest_indResult(statistic=-5.3084452807999245, pvalue=1.1582414092552812e-07)

In [21]:
#マンホイットニーのu検定
stats.mannwhitneyu( before_li, transition_period_li, alternative='two-sided')

MannwhitneyuResult(statistic=2489193.5, pvalue=0.020038774853821714)

In [22]:
stats.mannwhitneyu(transition_period_li, after_li, alternative='two-sided')

MannwhitneyuResult(statistic=2153349.0, pvalue=4.329536466197535e-23)

In [23]:
stats.mannwhitneyu(before_li, after_li, alternative='two-sided')

MannwhitneyuResult(statistic=2061966.0, pvalue=6.0414281978968784e-33)